In [6]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
data = pd.read_csv('Freq-Dataset.csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Freq-Dataset.csv'

In [4]:
data.describe()

,Number_of_Riders,Number_of_Drivers,Number_of_Past_Rides,Average_Ratings,Expected_Ride_Duration,Historical_Cost_of_Ride
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000
mean,60.372000,27.076000,50.031000,4.257220,99.58800,372.502623
std,23.701506,19.068346,29.313774,0.435781,49.16545,187.158756
min,20.000000,5.000000,0.000000,3.500000,10.00000,25.993449
25%,40.000000,11.000000,25.000000,3.870000,59.75000,221.365202
50%,60.000000,22.000000,51.000000,4.270000,102.00000,362.019426
75%,81.000000,38.000000,75.000000,4.632500,143.00000,510.497504
max,100.000000,89.000000,100.000000,5.000000,180.00000,836.116419


In [ ]:
import numpy as np

high_demand_percentile = 75
low_demand_percentile = 25

data['demand_multiplier'] = np.where(
    data['Number_of_Riders'] > np.percentile(data['Number_of_Riders'], high_demand_percentile),
    data['Number_of_Riders'] / np.percentile(data['Number_of_Riders'], high_demand_percentile),
    data['Number_of_Riders'] / np.percentile(data['Number_of_Riders'], low_demand_percentile))

high_supply_percentile = 75
low_supply_percentile = 25

data['supply_multiplier'] = np.where(
    data['Number_of_Drivers'] > np.percentile(data['Number_of_Drivers'], low_supply_percentile),
    np.percentile(data['Number_of_Drivers'], high_supply_percentile) / data['Number_of_Drivers'],
    np.percentile(data['Number_of_Drivers'], low_supply_percentile) / data['Number_of_Drivers'])

demand_threshold_high = 1.2
demand_threshold_low = 0.8
supply_threshold_high = 0.8
supply_threshold_low = 1.2

data['adjusted_ride_cost'] = data['Historical_Cost_of_Ride'] * (
        np.maximum(data['demand_multiplier'], demand_threshold_low) *
        np.maximum(data['supply_multiplier'], supply_threshold_high)
)

In [ ]:
data['profit_percentage'] = ((data['adjusted_ride_cost'] - data['Historical_Cost_of_Ride']) / data[
    'Historical_Cost_of_Ride']) * 100

profitable_rides = data[data['profit_percentage'] > 0]

loss_rides = data[data['profit_percentage'] < 0]

import plotly.graph_objects as go

profitable_count = len(profitable_rides)
loss_count = len(loss_rides)

labels = ['Profitable Rides', 'Loss Rides']
values = [profitable_count, loss_count]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=0.4)])
fig.update_layout(title='Profitability of Rides (Dynamic Pricing vs Historical Pricing)')
fig.show()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler


def data_preprocessing_pipeline(data):
    numeric_features = data.select_dtypes(include=['float', 'int']).columns
    categorical_features = data.select_dtypes(include=['object']).columns

    data[numeric_features] = data[numeric_features].fillna(data[numeric_features].mean())

    for feature in numeric_features:
        Q1 = data[feature].quantile(0.25)
        Q3 = data[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - (1.5 * IQR)
        upper_bound = Q3 + (1.5 * IQR)
        data[feature] = np.where((data[feature] < lower_bound) | (data[feature] > upper_bound),
                                 data[feature].mean(), data[feature])

        data[categorical_features] = data[categorical_features].fillna(data[categorical_features].mode().iloc[0])

        return data

In [ ]:
data["Vehicle_Type"] = data["Vehicle_Type"].map({"Premium": 1, "Economy": 0})

In [ ]:
data["Time_of_Booking"] = data["Time_of_Booking"].map({"Afternoon": 0, "Evening": 1, "Morning": 2, "Night": 3})

In [ ]:
from sklearn.model_selection import train_test_split

x = np.array(
    data[["Number_of_Riders", "Number_of_Drivers", "Vehicle_Type", "Time_of_Booking", "Expected_Ride_Duration"]])
y = np.array(data[["adjusted_ride_cost"]])

x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

y_train = y_train.ravel()
y_test = y_test.ravel()

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(x_train, y_train)

In [ ]:
def predict_price(number_of_riders, number_of_drivers, vehicle_type, time_of_booking, Expected_Ride_Duration):
    vehicle_type_numeric = get_vehicle_type_numeric(vehicle_type)
    if vehicle_type_numeric is None:
        raise ValueError("Invalid vehicle type")

    time_of_booking_numeric = get_time_of_booking_numeric(time_of_booking)
    if time_of_booking_numeric is None:
        raise ValueError("Invalid time of booking")

    input_data = np.array(
        [[number_of_riders, number_of_drivers, vehicle_type_numeric, time_of_booking_numeric, Expected_Ride_Duration]])
    predicted_price = model.predict(input_data)
    return predicted_price